In [1]:
import numpy as np
import sympy as sp
import itertools as it
import string
import pickle
from tqdm import tqdm
import time

In [2]:
#cyclic permutation to check equivalence under the parrtial trace
def cyclic_permutations(lst):
    yield lst
    for k in range(3, len(lst), 3):
        p = lst[k:] + lst[:k]
        if p == lst:
            break
        yield p

In [3]:
#cyclic permutation to check equivalence under the parrtial trace
def cyclic_permutations_clock(lst):
    yield lst
    for k in range(3, len(lst), 3):
        p = lst[-k:] + lst[:-k]
        if p == lst:
            break
        yield p

In [4]:
#the parties are defined as capital letters in alphabetical order, the inputs and outputs as numbers from 0 to n
parties = ['A']
inputs = ['0','1']
outputs = ['0','1']
operators = [''.join(k) for k in it.product(parties,outputs,inputs, repeat = 1)]
A_operators = [''.join(k) for k in it.product(parties,outputs,inputs, repeat = 1)]

In [5]:
#Add the shared state as 'R00' and the dual variable as 'Y00'
operators.append('R00')
variables = operators+['Y00']

In [6]:
#Define the level of the hierarchy to build the monomial set. Additional elements can be added manually
level = 2

monomial_set_rho_y = []

lev = []

for i in range(1,level+1):   
    
    lev = [''.join(k) for k in it.product(A_operators, repeat = i)]
    if(i==3):
        tmp= [''.join(k) for k in it.product(A_operators, repeat = 1)]
        for count in range(len(tmp)):
            for count1 in range(len(tmp)):
                #if(count<count1):
                lev.append('A00'+tmp[count]+tmp[count1])
                lev.append('A10'+tmp[count]+tmp[count1])
                lev.append(tmp[count]+'A00'+tmp[count1])
                #lev.append(tmp[count]+'A10'+tmp[count1])
                lev.append(tmp[count]+tmp[count1]+'A00')
                lev.append(tmp[count]+tmp[count1]+'A10')

            
            
            
    monomial_set_rho_y+=lev

#orthogonality
for k in range(len(monomial_set_rho_y)):
    for l1 in parties: 
        for l2 in parties:
            for i1 in inputs:
                for o1 in outputs:
                    for o2 in outputs:
                        if (o1!=o2 and l1+o1+i1+l2+o2+i1 in monomial_set_rho_y[k]):
                            monomial_set_rho_y[k] = ('')

#every element projects into itself
for p in range(len(monomial_set_rho_y)):      
    for m in range(1,int(i/2)+2):
        #s = [''.join(k) for k in it.product(operators, repeat = m)]
        while any(el+el in monomial_set_rho_y[p] for el in operators):
            for el in operators:
                monomial_set_rho_y[p] = monomial_set_rho_y[p].replace(el+el, el)


monomial_set_rho_y.append('')
monomial_set_rho_y = sorted(sorted(list(set(monomial_set_rho_y))), key=len)
print(len(monomial_set_rho_y))

13


In [7]:
monomial_set_rho_y

['',
 'A00',
 'A01',
 'A10',
 'A11',
 'A00A01',
 'A00A11',
 'A01A00',
 'A01A10',
 'A10A01',
 'A10A11',
 'A11A00',
 'A11A10']

In [8]:
n=3
rev_mon_set_rho_y = []

#len(monomial_set_rho)=len(monomial_set_y)
for l in range(len(monomial_set_rho_y)):
    lista = [monomial_set_rho_y[l][i:i+n] for i in range(0, len(monomial_set_rho_y[l]), n)]
    lista.reverse()
    rev_mon_set_rho_y.append(''.join(lista))

In [9]:
#Building the block matrices, no simplifications are performed yet.
gamma_mat_rho = []
gamma_mat_y = []

#len(monomial_set_rho)=len(monomial_set_y)
for i in range(len(monomial_set_rho_y)):
    gamma_row_rho = []
    for j in range(i, len(rev_mon_set_rho_y)):
        gamma_row_rho.append(monomial_set_rho_y[i]+'R00'+rev_mon_set_rho_y[j])
    gamma_mat_rho.append(gamma_row_rho)
for i in range(len(monomial_set_rho_y)):
    gamma_row_y = []
    for j in range(i, len(rev_mon_set_rho_y)):
        gamma_row_y.append(monomial_set_rho_y[i]+'Y00'+rev_mon_set_rho_y[j])
    gamma_mat_y.append(gamma_row_y)

In [10]:
#Application of the reduction rules to the first block matrix
for i in tqdm(range(len(gamma_mat_rho))):
    for j in range(len(gamma_mat_rho[i])):
        
        #setting orthogonal elements to 'O00'
        for l1 in parties: 
            for i1 in inputs:
                for o1 in outputs:
                    for o2 in outputs:
                        if (o1!=o2 and l1+o1+i1+l1+o2+i1 in gamma_mat_rho[i][j]):
                            gamma_mat_rho[i][j] = ('O00')

        #the partial trace is cyclical over the first party
        for l1 in parties: 
            for i1 in inputs:
                for o1 in outputs:
                    for o2 in outputs:
                        if (o1!=o2 and l1+o1+i1+l1+o2+i1 in gamma_mat_rho[i][j][0:3]+gamma_mat_rho[i][j][-3::]):
                            gamma_mat_rho[i][j] = ('O00')

        #every element in operators project into itself
        for m in range(1,int(len(gamma_mat_rho[i][j])/2)+1):
            while any(el+el in gamma_mat_rho[i][j] for el in operators):
                for el in operators:
                    gamma_mat_rho[i][j] = gamma_mat_rho[i][j].replace(el+el, el)
        
        #the partial trace is cyclical over the first party
        for m in range(1,int(len(gamma_mat_rho[i][j])/2)+1):
            while len(gamma_mat_rho[i][j])>7 and any(el+el in gamma_mat_rho[i][j][0:3]+gamma_mat_rho[i][j][-3::] for el in A_operators):
                gamma_mat_rho[i][j] = gamma_mat_rho[i][j][:-3]

        flag=0
        if(gamma_mat_rho[i][j][0:3] != 'R00'):
            for tmp in cyclic_permutations(gamma_mat_rho[i][j]):
                if (tmp[0:3] == 'R00'):
                    tmp1=tmp
                    tmp=gamma_mat_rho[i][j]
                    flag=1
                    break
        if(flag):
            gamma_mat_rho[i][j]=tmp1

diag_el_rho=[]
for i in range(len(gamma_mat_rho)):
    diag_el_rho.append(gamma_mat_rho[i][0])

100%|████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 2537.03it/s]


In [11]:
#Application of the reduction rules to the second block matrix
for i in tqdm(range(len(gamma_mat_y))):
    #time.sleep(0.01)
    for j in range(len(gamma_mat_y[i])):
        
        #setting orthogonal elements to 'O00'
        for l1 in parties: 
            for i1 in inputs:
                for o1 in outputs:
                    for o2 in outputs:
                        if (o1!=o2 and l1+o1+i1+l1+o2+i1 in gamma_mat_y[i][j]):
                            gamma_mat_y[i][j] = ('O00')

        #the partial trace is cyclical over the first party
        for l1 in parties: 
            for i1 in inputs:
                for o1 in outputs:
                    for o2 in outputs:
                        if (o1!=o2 and l1+o1+i1+l1+o2+i1 in gamma_mat_y[i][j][0:3]+gamma_mat_y[i][j][-3::]):
                            gamma_mat_y[i][j] = ('O00')

        #every element project into itself
        for m in range(1,int(len(gamma_mat_y[i][j])/2)+1):
            while any(el+el in gamma_mat_y[i][j] for el in operators):
                for el in operators:
                    gamma_mat_y[i][j] = gamma_mat_y[i][j].replace(el+el, el)
        
        #the partial trace is cyclical over the first party
        for m in range(1,int(len(gamma_mat_y[i][j])/2)+1):
            while len(gamma_mat_y[i][j])>7 and any(el+el in gamma_mat_y[i][j][0:3]+gamma_mat_y[i][j][-3::] for el in A_operators):
                gamma_mat_y[i][j] = gamma_mat_y[i][j][:-3]
        
        #the partial trace is cyclical over the first party
        if ('R00' not in gamma_mat_y[i][j]):
            for tmp in cyclic_permutations(gamma_mat_y[i][j]):
                if (tmp<gamma_mat_y[i][j]):
                    gamma_mat_y[i][j]=tmp
        else:
            flag=0
            if(gamma_mat_y[i][j][0:3] != 'R00'):
                for tmp in cyclic_permutations(gamma_mat_y[i][j]):
                    if (tmp[0:3] == 'R00'):
                        tmp1=tmp
                        tmp=gamma_mat_y[i][j]
                        flag=1
                        break
            if(flag):
                gamma_mat_y[i][j]=tmp1


#diag_el=diag_el+diag_el_rho+diag_el_y
#diag_el=sorted(sorted(set(diag_el)), key=len);

100%|██████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<?, ?it/s]


In [12]:
#Building the monomial index in the two block matrices
monomial_index_rho = {}
for i in range(len(gamma_mat_rho)):
    for j in range(len(gamma_mat_rho[i])):
        monomial_index_rho[gamma_mat_rho[i][j]] = []
for i in range(len(gamma_mat_rho)):
    for j in range(len(gamma_mat_rho[i])):
        monomial_index_rho[gamma_mat_rho[i][j]].append([i,i+j])
monomial_index_y = {}
for i in range(len(gamma_mat_y)):
    for j in range(len(gamma_mat_y[i])):
        monomial_index_y[gamma_mat_y[i][j]] = []
for i in range(len(gamma_mat_y)):
    for j in range(len(gamma_mat_y[i])):
        monomial_index_y[gamma_mat_y[i][j]].append([i,i+j])

In [13]:
diz_constraints_rho = { list(monomial_index_rho.keys())[i] : i  for i in range(len(list(monomial_index_rho.keys())))}
diz_constraints_y = { list(monomial_index_y.keys())[i] : i  for i in range(len(list(monomial_index_y.keys())))}

In [14]:
#The element 'O00' is not required
index_keys_rho = list(monomial_index_rho.keys())
index_keys_rho.remove('O00')
index_keys_y = list(monomial_index_y.keys())
index_keys_y.remove('O00')
index_keys=sorted(sorted(list(set(index_keys_rho+index_keys_y))), key=len)

In [15]:
#The variables are divided into real coefficient, real_C, complex coefficients, complex_C and hermitian and complex matrices, hermitian_M and complex_M
coefs = [item for item in index_keys if 'R00' not in item]
#coefs.remove('O00')
#coefs.remove('')
matrices = [item for item in index_keys if 'R00' in item]
hermitian_M=[]
complex_M=[]
real_C=[]
complex_C=[]


for i in matrices:
    lista = [i[j:j+n] for j in range(0, len(i), n)]
    lista.reverse()
    lista=''.join(lista)
    flag=0
    if(lista[0:3] != 'R00'):
        for tmp in cyclic_permutations(lista):
            if (tmp[0:3] == 'R00'):
                tmp1=tmp
                tmp=lista
                flag=1
                break
    if(flag):
        lista=tmp1
    if(lista == i):
        hermitian_M.append(i)


hermitian_M = sorted(sorted(set(hermitian_M)), key=len);
complex_M = [i for i in matrices if i not in hermitian_M]


for i in coefs:
    lista = [i[j:j+n] for j in range(0, len(i), n)]
    lista.reverse()
    lista = ''.join(lista)
    for tmp in cyclic_permutations(lista):
        if (tmp<lista):
            lista=tmp
    if(lista==i):
        real_C.append(i)


real_C=sorted(sorted(set(real_C)), key=len);
complex_C = [i for i in coefs if i not in real_C]

In [16]:
#Different indices have to be used for elements of the two block matrices
#The suffix index is used to identify the string corresponding to the target element in the lists;
#The suffix indices is used to identify the position of the string corresponding to the target element in the lists;
hermitian_rho_indices = []
complex_rho_indices = []
complex_mat_y_indices = []
hermitian_rho_index = []
complex_rho_index = []
hermitian_y_indices = []
complex_y_indices = []
real_y_indices = []
hermitian_y_index = []
complex_y_index = []
real_y_index = []
complex_mat_y_index = []

list_of_indices=real_C+complex_C+hermitian_M+complex_M
#index_keys_rho.remove('')
#index_keys_y.remove('')
for i in index_keys_rho:
    if i in hermitian_M:
        hermitian_rho_indices.append(hermitian_M.index(i)+1)
        hermitian_rho_index.append(i)
    else:
        complex_rho_indices.append(complex_M.index(i)+1)
        complex_rho_index.append(i)
for i in index_keys_y:
    if i in hermitian_M:
        hermitian_y_indices.append(hermitian_M.index(i)+1)
        hermitian_y_index.append(i)
    elif i in complex_M:
        complex_mat_y_indices.append(complex_M.index(i)+1)
        complex_mat_y_index.append(i)
    elif i in real_C:
        real_y_indices.append(real_C.index(i)+1)
        real_y_index.append(i)
    elif i in complex_C:
        complex_y_indices.append(complex_C.index(i)+1)
        complex_y_index.append(i)


In [17]:
#Here the final list containing the position of the target strings in the final gamma matrix is provided;
A_rho = np.zeros([len(hermitian_rho_index),len(monomial_set_rho_y),len(monomial_set_rho_y)], dtype=int)
for i in range(len(hermitian_rho_index)):
    for j in range(len(monomial_index_rho[hermitian_rho_index[i]])):
        A_rho[i][monomial_index_rho[hermitian_rho_index[i]][j][0]][monomial_index_rho[hermitian_rho_index[i]][j][1]] = 1
for i in range(len(hermitian_rho_index)):
    A_rho[i]=A_rho[i]+A_rho[i].T
sdp_matrices_rho=A_rho.astype(bool)
A_rho = np.zeros([len(complex_rho_index),len(monomial_set_rho_y),len(monomial_set_rho_y)], dtype=int)
for i in range(len(complex_rho_index)):
    for j in range(len(monomial_index_rho[complex_rho_index[i]])):
        A_rho[i][monomial_index_rho[complex_rho_index[i]][j][0]][monomial_index_rho[complex_rho_index[i]][j][1]] = 1
for i in range(len(complex_rho_index)):
    A_rho[i]=A_rho[i]+A_rho[i].T
complex_matrices_rho=A_rho.astype(bool)
A_y = np.zeros([len(real_y_index),len(monomial_set_rho_y),len(monomial_set_rho_y)], dtype=int)
for i in range(len(real_y_index)):
    for j in range(len(monomial_index_y[real_y_index[i]])):
        A_y[i][monomial_index_y[real_y_index[i]][j][0]][monomial_index_y[real_y_index[i]][j][1]] = 1
for i in range(len(real_y_index)):
    A_y[i]=A_y[i]+A_y[i].T
coefs_y=A_y.astype(bool)
A_y = np.zeros([len(complex_y_index),len(monomial_set_rho_y),len(monomial_set_rho_y)], dtype=int)
for i in range(len(complex_y_index)):
    for j in range(len(monomial_index_y[complex_y_index[i]])):
        A_y[i][monomial_index_y[complex_y_index[i]][j][0]][monomial_index_y[complex_y_index[i]][j][1]] = 1
for i in range(len(complex_y_index)):
    A_y[i]=A_y[i]+A_y[i].T
complex_coefs_y=A_y.astype(bool)
A_y = np.zeros([len(hermitian_y_index),len(monomial_set_rho_y),len(monomial_set_rho_y)], dtype=int)
for i in range(len(hermitian_y_index)):
    for j in range(len(monomial_index_y[hermitian_y_index[i]])):
        A_y[i][monomial_index_y[hermitian_y_index[i]][j][0]][monomial_index_y[hermitian_y_index[i]][j][1]] = 1
for i in range(len(hermitian_y_index)):
    A_y[i]=A_y[i]+A_y[i].T
sdp_matrices_y=A_y.astype(bool)
A_y = np.zeros([len(complex_mat_y_index),len(monomial_set_rho_y),len(monomial_set_rho_y)], dtype=int)
for i in range(len(complex_mat_y_index)):
    for j in range(len(monomial_index_y[complex_mat_y_index[i]])):
        A_y[i][monomial_index_y[complex_mat_y_index[i]][j][0]][monomial_index_y[complex_mat_y_index[i]][j][1]] = 1
for i in range(len(complex_mat_y_index)):
    A_y[i]=A_y[i]+A_y[i].T
complex_matrices_y=A_y.astype(bool)


In [18]:
complex_C_pairs=[]
for i in complex_C:
    lista = [i[j:j+n] for j in range(0, len(i), n)]
    lista.reverse()
    lista=''.join(lista)
    for tmp in cyclic_permutations(lista):
        if (tmp<lista):
            lista=tmp
    if(lista in complex_C):
        complex_C_pairs.append([complex_C.index(i), complex_C.index(lista)])
for i in range(len(complex_C_pairs)):
    complex_C_pairs[i]=sorted(complex_C_pairs[i])
complex_C_pairs=np.unique(complex_C_pairs, axis=0)

complex_M_pairs=[]
for i in complex_M:
    lista = [i[j:j+n] for j in range(0, len(i), n)]
    lista.reverse()
    lista=''.join(lista)
    flag=0
    if(lista[0:3] != 'R00'):
        for tmp in cyclic_permutations(lista):
            if (tmp[0:3] == 'R00'):
                tmp1=tmp
                tmp=lista
                flag=1
                break
    if(flag):
        lista=tmp1
    if(lista in complex_M):
        complex_M_pairs.append([complex_M.index(i), complex_M.index(lista)])
print(len(complex_M_pairs))
for i in range(len(complex_M_pairs)):
    complex_M_pairs[i]=sorted(complex_M_pairs[i])
complex_M_pairs=np.unique(complex_M_pairs, axis=0)


16


In [19]:
tracial_pairs=[]
for item in matrices:
    if(len(item)>7):
        if(item[:3]+item[-3:]=='R00R00'):
            tmp_1=0
            tmp_2=0
            if(item in complex_M):
                tmp_1 = 1
                index_1 = complex_M.index(item)+1
                if(item[:-3] in complex_M):
                    tmp_2=1
                    index_2=complex_M.index(item[:-3])+1
                else:
                    index_2=hermitian_M.index(item[:-3])
            else:
                index_1 = hermitian_M.index(item)
                if(item[:-3] in complex_M):
                    tmp_2=1
                    index_2=complex_M.index(item[:-3])+1
                else:
                    index_2=hermitian_M.index(item[:-3])
            tracial_pairs.append([tmp_1, tmp_2, index_1, index_2])

In [20]:
var_num=[len(real_C), len(hermitian_M), len(complex_C), len(complex_M)]

In [21]:
from scipy.io import savemat

In [22]:
savemat("loc_fidelity_d3_N2_lvl_2_test.mat", {'var_num': var_num, 
                                         'complex_matrices_rho': complex_matrices_rho, 
                                         'sdp_matrices_rho': sdp_matrices_rho, 'sdp_matrices_y': sdp_matrices_y, 
                                         'coefs_y': coefs_y, 'complex_coefs_y': complex_coefs_y,
                                         'complex_matrices_y': complex_matrices_y, 
                                         'rho_indices': hermitian_rho_indices, 
                                         'complex_rho_indices': complex_rho_indices, 
                                         'hermitian_y_indices': hermitian_y_indices, 
                                         'complex_mat_y_indices': complex_mat_y_indices, 
                                         'real_y_indices': real_y_indices, 
                                         'complex_y_indices': complex_y_indices,
                                         'complex_C_pairs':complex_C_pairs,
                                         'complex_M_pairs':complex_M_pairs,
                                         'tracial_pairs':tracial_pairs})

In [23]:
var_num

[13, 13, 32, 32]

In [26]:
gamma_mat_rho

[['R00',
  'R00A00',
  'R00A01',
  'R00A10',
  'R00A11',
  'R00A01A00',
  'R00A11A00',
  'R00A00A01',
  'R00A10A01',
  'R00A01A10',
  'R00A11A10',
  'R00A00A11',
  'R00A10A11'],
 ['R00A00',
  'R00A01A00',
  'O00',
  'R00A11A00',
  'R00A01A00',
  'R00A11A00',
  'R00A00A01A00',
  'R00A10A01A00',
  'O00',
  'O00',
  'R00A00A11A00',
  'R00A10A11A00'],
 ['R00A01',
  'R00A10A01',
  'O00',
  'R00A01A00A01',
  'R00A11A00A01',
  'R00A00A01',
  'R00A10A01',
  'R00A01A10A01',
  'R00A11A10A01',
  'O00',
  'O00'],
 ['R00A10',
  'R00A11A10',
  'O00',
  'O00',
  'R00A00A01A10',
  'R00A10A01A10',
  'R00A01A10',
  'R00A11A10',
  'R00A00A11A10',
  'R00A10A11A10'],
 ['R00A11',
  'R00A01A00A11',
  'R00A11A00A11',
  'O00',
  'O00',
  'R00A01A10A11',
  'R00A11A10A11',
  'R00A00A11',
  'R00A10A11'],
 ['R00A01A00A01',
  'R00A11A00A01',
  'R00A00A01A00A01',
  'R00A10A01A00A01',
  'O00',
  'O00',
  'R00A00A11A00A01',
  'R00A10A11A00A01'],
 ['R00A11A00A11',
  'R00A00A01A00A11',
  'R00A10A01A00A11',
  'O00',
  'O

In [28]:
complex_C

['A00A01Y00',
 'A00A11Y00',
 'A00Y00A01',
 'A00Y00A11',
 'A01A10Y00',
 'A01Y00A10',
 'A10A11Y00',
 'A10Y00A11',
 'A00A01A10Y00',
 'A00A01Y00A11',
 'A00A11A10Y00',
 'A00A11Y00A01',
 'A00Y00A10A01',
 'A00Y00A10A11',
 'A01A10A11Y00',
 'A01Y00A11A10',
 'A00A01A00A01Y00',
 'A00A01A00A11Y00',
 'A00A01Y00A00A11',
 'A00A01Y00A10A01',
 'A00A01Y00A10A11',
 'A00A11A00A11Y00',
 'A00A11A10A01Y00',
 'A00A11A10A11Y00',
 'A00A11Y00A10A01',
 'A00A11Y00A10A11',
 'A00Y00A01A10A01',
 'A00Y00A11A10A01',
 'A01A10A01A10Y00',
 'A01A10Y00A11A10',
 'A01Y00A10A11A10',
 'A10A11A10A11Y00']

In [29]:
real_C

['Y00',
 'A00Y00',
 'A01Y00',
 'A10Y00',
 'A11Y00',
 'A00A01A00Y00',
 'A00A01Y00A01',
 'A00A11A00Y00',
 'A00A11Y00A11',
 'A01A10A01Y00',
 'A01A10Y00A10',
 'A10A11A10Y00',
 'A10A11Y00A11']